<center><h1>第七章 缺失数据</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 一、缺失值的统计和删除
### 1. 缺失信息的统计

缺失数据可以使用`isna`或`isnull`（两个函数没有区别）来查看每个单元格是否缺失，结合`mean`可以计算出每列缺失值的比例：

In [2]:
df = pd.read_csv('data/learn_pandas.csv', usecols=['Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer'])
df.isna().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [3]:
df.isna().mean()

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

如果想要查看某一列缺失或者非缺失的行，可以利用`Series`上的`isna`或者`notna`进行布尔索引。例如，查看身高缺失的行：

In [4]:
df[df.Height.isna()].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
12,Senior,Peng You,Female,NaN,48.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N
36,Freshman,Xiaojuan Qin,Male,NaN,79.0,Y
60,Freshman,Yanpeng Lv,Male,NaN,65.0,N


如果想要同时对几个列，检索出全部为缺失或者至少有一个缺失或者没有缺失的行，可以使用`isna, notna`和`any, all`的组合。例如，对身高、体重和转系情况这3列分别进行这三种情况的检索：

In [5]:
sub_set = df[['Height', 'Weight', 'Transfer']]
df[sub_set.isna().all(axis=1)] # 全部缺失

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


In [6]:
df[sub_set.isna().any(1)].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N
12,Senior,Peng You,Female,NaN,48.0,NaN
21,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N


In [7]:
df[sub_set.notna().all(1)].head() # 没有缺失

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


### 2. 缺失信息的删除

数据处理中经常需要根据缺失值的大小、比例或其他特征来进行行样本或列特征的删除，`pandas`中提供了`dropna`函数来进行操作。

`dropna`的主要参数为轴方向`axis`（默认为0，即删除行）、删除方式`how`、删除的非缺失值个数阈值`thresh`（$\color{red}{非缺失值}$没有达到这个数量的相应维度会被删除）、备选的删除子集`subset`，其中`how`主要有`any`和`all`两种参数可以选择。

例如，删除身高体重至少有一个缺失的行：

In [8]:
res = df.dropna(how='any', subset=['Height', 'Weight'])
res.shape

(174, 6)

例如，删除超过15个缺失值的列：

In [9]:
res = df.dropna(1, thresh=df.shape[0]-15) # 身高被删除
res.head()

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


当然，不用`dropna`同样是可行的，例如上述的两个操作，也可以使用布尔索引来完成：

In [10]:
res = df.loc[df[['Height', 'Weight']].notna().all(1)]
res.shape

(174, 6)

In [11]:
res = df.loc[:, ~(df.isna().sum() > 15)]
res.head()

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


In [12]:
df[['Height', 'Weight']].notna().all(1)

0       True
1       True
2       True
3      False
4       True
       ...  
195     True
196     True
197     True
198     True
199     True
Length: 200, dtype: bool

## 二、缺失值的填充和插值
### 1. 利用fillna进行填充

在`fillna`中有三个参数是常用的：`value, method, limit`。其中，`value`为填充值，可以是标量，也可以是索引到元素的字典映射；`method`为填充方法，有用前面的元素填充`ffill`和用后面的元素填充`bfill`两种类型，`limit`参数表示连续缺失值的最大填充次数。

下面构造一个简单的`Series`来说明用法：

In [13]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan], list('aaabcd'))
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [14]:
s.fillna(method='ffill') # 用前面的值向后填充

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [15]:
s.fillna(method='ffill', limit=1) # 连续出现的确实，最多填充一次

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [16]:
s.fillna(s.mean()) # value为标量

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [17]:
s.fillna({'a':100, 'b':200}) # 通过索引映射填充的值

a    100.0
a      1.0
a    100.0
b    200.0
c      2.0
d      NaN
dtype: float64

有时为了更加合理地填充，需要先进行分组后再操作。例如，根据年级进行身高的均值填充：

In [18]:
df.groupby('Grade')['Height'].transform(lambda x : x.fillna(x.mean()))

0      158.900000
1      166.500000
2      188.900000
3      163.075862
4      174.000000
          ...    
195    153.900000
196    160.900000
197    153.900000
198    175.300000
199    155.700000
Name: Height, Length: 200, dtype: float64

#### 【练一练】
对一个序列以如下规则填充缺失值：如果单独出现的缺失值，就用前后均值填充，如果连续出现的缺失值就不填充，即序列`[1, NaN, 3, NaN, NaN]`填充后为`[1, 2, 3, NaN, NaN]`，请利用`fillna`函数实现。（提示：利用`limit`参数）
#### 【END】


In [23]:
s = pd.Series([np.nan, 1, np.nan, 2, np.nan, np.nan])
(s.fillna(method='ffill', limit=1) + s.fillna(method='bfill', limit=1)) / 2

# nan加其它数还为nan的特性

0    NaN
1    1.0
2    1.5
3    2.0
4    NaN
5    NaN
dtype: float64

### 2. 插值函数

在关于`interpolate`函数的[文档](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html#pandas.Series.interpolate)描述中，列举了许多插值法，包括了大量`Scipy`中的方法。由于很多插值方法涉及到比较复杂的数学知识，因此这里只讨论比较常用且简单的三类情况，即线性插值、最近邻插值和索引插值。

对于`interpolate`而言，除了插值方法（默认为`linear`线性插值）之外，有与`fillna`类似的两个常用参数，一个是控制方向的`limit_direction`，另一个是控制最大连续缺失值插值个数的`limit`。其中，限制插值的方向默认为`forward`，这与`fillna`的`method`中的`ffill`是类似的，若想要后向限制插值或者双向限制插值可以指定为`backward`或`both`。

In [26]:
s = pd.Series([np.nan, np.nan, 1, np.nan, np.nan, np.nan, 2, np.nan, np.nan])
s.values

array([nan, nan,  1., nan, nan, nan,  2., nan, nan])

例如，在默认线性插值法下分别进行`backward`和双向限制插值，同时限制最大连续条数为1：

In [28]:
res = s.interpolate(limit_direction='backward', limit=1)
res.values

array([ nan, 1.  , 1.  ,  nan,  nan, 1.75, 2.  ,  nan,  nan])